Set Shots - Expected Goal Model - Model Evaluation - GBM

In [1]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/")

In [44]:
import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_score_model.config import set_shots_file_path
# from total_points_model.domain.preprocessing.data_preprocessor import DataPreprocessor
# from total_points_model.domain.contracts.mappings import Mappings
from expected_score_model.domain.contracts.modelling_data_contract import ModellingDataContract
from expected_score_model.domain.modelling.supermodel import SuperXGBClassifier
from expected_score_model.domain.modelling.model_evaluation import XGBClassifierEvaluator

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


New Model

In [3]:
new_version = 2
model_name = "expected_goal_set"
model_file_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models"
preproc_file_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors"

Comparison Model

In [4]:
old_version = new_version - 1
compare_train_predictions_file_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/predictions/train_predictions_"+model_name+"_v"+str(old_version)+".csv"
compare_test_predictions_file_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/predictions/test_predictions_"+model_name+"_v"+str(old_version)+".csv"

Response

In [5]:
RESPONSE = ModellingDataContract.RESPONSE_GOAL

Load Data

In [6]:
df_modelling = pd.read_csv(set_shots_file_path)
df_modelling.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,random100,ModellingFilter
11052,208,possGain,behind,1663.0,4,634,683.0,Geelong,Geelong,Tom Hawkins,Tom_Hawkins,Kick,41.0,25.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,0,1,1,1,0,106.0,33.0,73.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,41.0,48.0,48.0,26.0,25.0,21.0,21.0,14.0,683.0,644.0,642.0,641.0,49.0,39.0,8.062258,39.0,25.0,46.324939,0.570040,32.660913,0.116867,6.695990,0.364585,20.889203,1521.0,3.663562,True,10,True
11053,212,kickIn,goal,1730.0,4,787,885.0,Geelong,Geelong,Patrick Dangerfield,Patrick_Dangerfield,Kick,51.0,43.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,0,1,1,1,0,108.0,34.0,74.0,Kick,Contested Mark,Contest Target,Kick Inside 50 Result,51.0,53.0,53.0,53.0,43.0,38.0,38.0,38.0,885.0,852.0,851.0,850.0,98.0,33.0,5.385165,29.0,43.0,51.865210,0.977446,56.003541,0.069260,3.968302,0.214332,12.280314,841.0,3.367296,True,66,True
11054,213,centreBounce,goal,1754.0,4,956,979.0,Sydney,Sydney,Paddy McCartin,Paddy_McCartin,Kick,66.0,0.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,6,0,6,115.0,34.0,81.0,Kick,Free For,Kick Inside 50 Result,Kick Into F50,66.0,74.0,74.0,27.0,0.0,0.0,0.0,17.0,979.0,977.0,968.0,967.0,23.0,2.0,8.000000,14.0,0.0,14.000000,0.000000,0.000000,0.482343,27.636207,2.588598,148.315731,196.0,2.639057,True,12,True
11055,219,centreBounce,goal,1846.0,4,1252,1266.0,Sydney,Sydney,Chad Warner,Chad_Warner,Kick,26.0,3.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,6,0,6,121.0,40.0,81.0,Kick,Uncontested Mark,Ground Kick,Handball,26.0,18.0,-5.0,-2.0,3.0,2.0,-4.0,3.0,1266.0,1265.0,1264.0,1260.0,14.0,1.0,8.062258,54.0,3.0,54.083269,0.055499,3.179830,0.118569,6.793500,0.365993,20.969862,2916.0,3.988984,True,18,True
11056,228,centreBounce,goal,1912.0,4,1519,1560.0,Geelong,Geelong,Jeremy Cameron,Jeremy_Cameron,Kick,41.0,21.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,6,6,0,127.0,46.0,81.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,41.0,49.0,49.0,25.0,21.0,17.0,17.0,33.0,1560.0,1535.0,1532.0,1531.0,41.0,25.0,8.944272,39.0,21.0,44.294469,0.493941,28.300756,0.127885,7.327246,0.400462,22.944787,1521.0,3.663562,True,93,False


In [7]:
training_data = df_modelling[df_modelling[ModellingDataContract.TRAIN_TEST_SPLIT_COL]]
test_data = df_modelling[~df_modelling[ModellingDataContract.TRAIN_TEST_SPLIT_COL]]

In [8]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]

In [9]:
X_test.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Behind,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,random100,ModellingFilter
6,72,possGain,goal,696.0,2,462,506.0,Sydney,Sydney,Isaac Heeney,Isaac_Heeney,Kick,47.0,35.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,6,0,6,20.0,28.0,-8.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,47.0,53.0,53.0,42.0,35.0,27.0,27.0,41.0,506.0,468.0,466.0,465.0,44.0,38.0,10.000000,31.0,35.0,46.754679,0.845930,48.468229,0.091187,5.224603,0.284262,16.287036,961.0,3.433987,True,90,False
10,108,possGain,goal,982.0,2,1782,1859.0,Sydney,Sydney,Sam Reid,Sam_Reid,Kick,40.0,36.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,6,0,6,50.0,46.0,4.0,Kick,Uncontested Mark,Kick,Uncontested Mark,40.0,44.0,43.0,50.0,36.0,32.0,48.0,40.0,1859.0,1835.0,1833.0,1814.0,77.0,24.0,5.656854,38.0,36.0,52.345009,0.758378,43.451842,0.089092,5.104600,0.275545,15.787586,1444.0,3.637586,True,95,False
27,15,ballUp,behind,140.0,1,453,464.0,Collingwood,Collingwood,Brody Mihocek,Brody_Mihocek,Kick,31.0,-23.0,ineffective,True,NaN,141,160,Collingwood,Western Bulldogs,right,202101_Collingwood_WesternBulldogs,202101,2021,1,1,1,0,0.0,18.0,-18.0,Kick,Mark On Lead,Kick,Bounce,31.0,34.0,-7.0,-8.0,-23.0,-32.0,-5.0,-5.0,464.0,461.0,460.0,460.0,11.0,3.0,9.486833,49.0,23.0,54.129474,0.438859,25.144786,0.107406,6.153916,0.331520,18.994682,2401.0,3.891820,True,96,False
39,156,throwIn,miss,1357.0,3,766,816.0,Western Bulldogs,Western Bulldogs,Josh Bruce,Josh_Bruce,Kick,42.0,38.0,clanger,True,NaN,141,160,Collingwood,Western Bulldogs,right,202101_Collingwood_WesternBulldogs,202101,2021,0,0,0,0,36.0,48.0,-12.0,Kick,Free For,Kick Inside 50 Result,Kick Into F50,42.0,46.0,46.0,-1.0,38.0,34.0,34.0,61.0,816.0,784.0,772.0,771.0,50.0,32.0,5.656854,38.0,38.0,53.740115,0.785398,45.000000,0.084510,4.842076,0.260959,14.951857,1444.0,3.637586,True,91,False
41,195,possGain,miss,1617.0,3,1666,1697.0,Western Bulldogs,Western Bulldogs,Aaron Naughton,Aaron_Naughton,Kick,46.0,-44.0,ineffective,True,NaN,141,160,Collingwood,Western Bulldogs,right,202101_Collingwood_WesternBulldogs,202101,2021,1,1,0,1,36.0,62.0,-26.0,Kick,Mark On Lead,Kick,Handball Received,46.0,46.0,14.0,10.0,-44.0,-42.0,-32.0,-28.0,1697.0,1670.0,1668.0,1667.0,31.0,27.0,2.000000,34.0,44.0,55.605755,0.912908,52.305760,0.070609,4.045598,0.217612,12.468227,1156.0,3.526361,True,81,False


Preprocess Data

In [10]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train_preproc = preprocessor.transform(X_train)
# X_test_preproc = preprocessor.transform(X_test)

In [11]:
X_train_preproc = X_train[ModellingDataContract.modelling_feature_list]
X_test_preproc = X_test[ModellingDataContract.modelling_feature_list]

In [12]:
X_train_preproc.shape, X_test_preproc.shape

((8729, 2), (2328, 2))

In [13]:
X_train_preproc.head()

,Distance_to_Middle_Goal,Angle_to_Middle_Goal
0,52.009614,0.908067
1,43.600459,0.931882
2,36.674242,0.451453
3,33.015148,0.957589
4,46.572524,0.260602


In [14]:
X_test_preproc.head()

,Distance_to_Middle_Goal,Angle_to_Middle_Goal
6,46.754679,0.845930
10,52.345009,0.758378
27,54.129474,0.438859
39,53.740115,0.785398
41,55.605755,0.912908


Load Model

In [15]:
xgb_model = joblib.load(model_file_path + "/" + model_name + "_v" + str(new_version) + ".joblib")

Get Predictions

In [16]:
train_preds = xgb_model.predict(X_train_preproc)
test_preds = xgb_model.predict(X_test_preproc)

In [17]:
train_probas = xgb_model.predict_proba(X_train_preproc)[:, 1]
test_probas = xgb_model.predict_proba(X_test_preproc)[:, 1]

Compare Model

In [18]:
compare_train_data = pd.read_csv(compare_train_predictions_file_path)
compare_test_data = pd.read_csv(compare_test_predictions_file_path)

In [19]:
compare_train_data = compare_train_data.rename(columns = {'xgb_preds':'old_preds',
                                                          'xgb_probas':'old_probas'})
compare_test_data = compare_test_data.rename(columns = {'xgb_preds':'old_preds',
                                                        'xgb_probas':'old_probas'})


In [21]:
training_data.shape, compare_train_data.shape

((8729, 66), (8729, 7))

In [22]:
training_data = pd.merge(training_data, compare_train_data[["Match_ID", "Chain_Number", "Order", 'old_preds', 'old_probas']], how = "left", on = ["Match_ID", "Chain_Number", "Order"])
test_data = pd.merge(test_data, compare_test_data[["Match_ID", "Chain_Number", 'Order', 'old_preds', 'old_probas']], how = "left", on = ["Match_ID", "Chain_Number", "Order"])

In [23]:
training_data.shape

(8831, 68)

In [45]:
training_data['Match_ID'].value_counts().sort_index() - compare_train_data['Match_ID'].value_counts().sort_index()

202101_BrisbaneLions_Sydney                    0
202101_Collingwood_WesternBulldogs             0
202101_Essendon_Hawthorn                       0
202101_GreaterWesternSydney_StKilda            0
202101_Melbourne_Fremantle                     0
202101_NorthMelbourne_PortAdelaide             0
202101_Richmond_Carlton                        0
202101_WestCoast_GoldCoast                     2
202102_Carlton_Collingwood                     0
202102_Fremantle_GreaterWesternSydney          0
202102_Geelong_BrisbaneLions                   0
202102_GoldCoast_NorthMelbourne                0
202102_Hawthorn_Richmond                       0
202102_PortAdelaide_Essendon                   0
202102_StKilda_Melbourne                       0
202102_Sydney_Adelaide                         0
202102_WesternBulldogs_WestCoast               0
202103_Adelaide_GoldCoast                      0
202103_Carlton_Fremantle                       0
202103_Collingwood_BrisbaneLions               0
202103_Essendon_StKi

In [47]:
compare_train_data[compare_train_data['Match_ID'] == "202101_WestCoast_GoldCoast"]

,Match_ID,Chain_Number,Goal,Distance_to_Middle_Goal,Angle_to_Middle_Goal,old_preds,old_probas
154,202101_WestCoast_GoldCoast,7,1,22.852790,0.175940,1,0.886740
155,202101_WestCoast_GoldCoast,9,0,47.563116,0.211828,1,0.512136
156,202101_WestCoast_GoldCoast,12,1,32.882366,0.152649,1,0.761703
157,202101_WestCoast_GoldCoast,49,0,51.295711,0.750929,0,0.190474
158,202101_WestCoast_GoldCoast,51,0,44.059619,1.086847,0,0.162157
159,202101_WestCoast_GoldCoast,58,0,54.555018,0.336242,0,0.257997
160,202101_WestCoast_GoldCoast,77,0,47.793828,0.733593,0,0.280642
161,202101_WestCoast_GoldCoast,80,1,29.261750,0.348771,1,0.706860
162,202101_WestCoast_GoldCoast,96,1,38.512985,0.515926,1,0.580335
163,202101_WestCoast_GoldCoast,109,1,34.003676,0.298499,1,0.706860


In [46]:
training_data[training_data['Match_ID'] == "202101_WestCoast_GoldCoast"]

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,random100,ModellingFilter,old_preds,old_probas
154,7,possGain,goal,44.0,1,218,258.0,West Coast,West Coast,Oscar Allen,Oscar_Allen,Kick,60.0,4.0,effective,True,NaN,130,165,West Coast,Gold Coast,left,202101_WestCoast_GoldCoast,202101,2021,1,0,6,6,0,12.0,0.0,12.0,Kick,Free For,Contest Target,Kick Inside 50 Result,60.0,63.0,63.0,63.0,4.0,1.0,1.0,1.0,258.0,227.0,223.0,222.0,40.0,31.0,4.242641,22.5,4.0,22.852790,0.175940,10.080598,0.281245,16.114123,1.004441,57.550226,506.25,3.113515,True,75,True,1,0.886740
155,9,kickIn,behind,73.0,1,363,408.0,West Coast,West Coast,Jamie Cripps,Jamie_Cripps,Kick,36.0,10.0,ineffective,True,NaN,130,165,West Coast,Gold Coast,left,202101_WestCoast_GoldCoast,202101,2021,0,1,1,1,0,18.0,1.0,17.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,36.0,43.0,43.0,23.0,10.0,9.0,9.0,36.0,408.0,379.0,377.0,376.0,45.0,29.0,7.071068,46.5,10.0,47.563116,0.211828,12.136824,0.132149,7.571558,0.411412,23.572143,2162.25,3.839452,True,61,True,1,0.512136
156,12,possGain,goal,124.0,1,505,548.0,Gold Coast,Gold Coast,Ben King,Ben_King,Kick,50.0,5.0,effective,True,NaN,130,165,West Coast,Gold Coast,left,202101_WestCoast_GoldCoast,202101,2021,1,0,6,0,6,19.0,1.0,18.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,50.0,60.0,60.0,45.0,5.0,3.0,3.0,42.0,548.0,516.0,515.0,515.0,43.0,32.0,10.198039,32.5,5.0,32.882366,0.152649,8.746162,0.194209,11.127368,0.630883,36.146929,1056.25,3.481240,True,24,True,1,0.761703
157,49,possGain,miss,490.0,1,1573,1610.0,West Coast,West Coast,Liam Ryan,Liam_Ryan,Kick,45.0,-35.0,ineffective,True,NaN,130,165,West Coast,Gold Coast,left,202101_WestCoast_GoldCoast,202101,2021,0,1,1,1,0,20.0,16.0,4.0,Kick,Uncontested Mark,Kick Inside 50 Result,Kick Into F50,45.0,51.0,51.0,14.0,-35.0,-30.0,-30.0,-40.0,1610.0,1575.0,1574.0,1574.0,37.0,35.0,7.810250,37.5,35.0,51.295711,0.750929,43.025066,0.091568,5.246446,0.283566,16.247144,1406.25,3.624341,True,38,True,0,0.190474
158,51,possGain,behind,504.0,1,1645,1684.0,West Coast,West Coast,Josh J. Kennedy,Josh_J._Kennedy,Kick,62.0,-39.0,ineffective,True,NaN,130,165,West Coast,Gold Coast,left,202101_WestCoast_GoldCoast,202101,2021,0,1,1,1,0,21.0,16.0,5.0,Kick,Mark On Lead,Kick Inside 50 Result,Kick Into F50,62.0,64.0,64.0,47.0,-39.0,-34.0,-34.0,-46.0,1684.0,1651.0,1649.0,1649.0,39.0,33.0,5.385165,20.5,39.0,44.059619,1.086847,62.271743,0.067944,3.892888,0.212861,12.196063,420.25,3.020425,True,19,True,0,0.162157
159,58,possGain,miss,561.0,2,101,124.0,Gold Coast,Gold Coast,Lachie Weller,Lachie_Weller,Kick,31.0,18.0,clanger,True,NaN,130,165,West Coast,Gold Coast,left,202101_WestCoast_GoldCoast,202101,2021,0,0,0,0,0,22.0,16.0,6.0,Kick,Free For,Handball,Handball Received,31.0,30.0,2.0,-6.0,18.0,15.0,0.0,-4.0,124.0,116.0,110.0,108.0,23.0,8.0,3.162278,51.5,18.0,54.555018,0.336242,19.265264,0.111126,6.367035,0.342846,19.643654,2652.25,3.941582,True,28,True,0,0.257997
160,77,possGain,behind,739.0,2,660,746.0,West Coast,West Coast,Andrew Gaff,Andrew_Gaff,Kick,47.0,32.0,ineffective,True,NaN,130,165,West Coast,Gold Coast,left,202101_WestCoast_GoldCoast,202101,2021,0,1,1,1,0,28.0,23.0,5.0,Kick,Uncontested Mark,Kick,Free For

Evaluate Model

In [ ]:
training_data['xgb_preds'] = train_preds
training_data['xgb_probas'] = train_probas

train_xgb_evals = XGBClassifierEvaluator(model = xgb_model,
                                       data = training_data,
                                       actual_name = RESPONSE,
                                       expected_name = "xgb_probas",
                                       expected_label_name = "xgb_preds",
                                       compare_name='old_probas',
                                       compare_label_name="old_preds"
                                       )

In [ ]:
test_data['xgb_preds'] = test_preds
test_data['xgb_probas'] = test_probas

test_xgb_evals = XGBClassifierEvaluator(model = xgb_model,
                                       data = test_data,
                                       actual_name = RESPONSE,
                                       expected_name = "xgb_probas",
                                       expected_label_name = "xgb_preds",
                                       # compare_name = "old_preds",
                                       # compare_label_name="old_preds"
                                       )

Model Averages

In [ ]:
train_xgb_evals.data[[RESPONSE, 'xgb_preds']].describe()

In [ ]:
test_xgb_evals.data[[RESPONSE, 'xgb_preds']].describe()

Plots

In [ ]:
train_xgb_evals.plot_feature_importance(max_num_features=10, importance_type="total_gain")
test_xgb_evals.plot_feature_importance(max_num_features=10, importance_type="total_gain")

In [ ]:
train_xgb_evals.plot_shap_summary_plot()
test_xgb_evals.plot_shap_summary_plot()

In [ ]:
top_features = test_xgb_evals.get_ranked_feature_importance()
top_10_features = top_features[:10]
top_10_features

In [ ]:
for col in top_10_features:
    train_xgb_evals.plot_pdp([col])
    test_xgb_evals.plot_pdp([col])

In [ ]:
for col in top_10_features:
    train_xgb_evals.plot_ice([col])
    test_xgb_evals.plot_ice([col])

In [ ]:
for col in top_10_features:
    train_xgb_evals.plot_feature_ave(col)
    test_xgb_evals.plot_feature_ave(col)

Model Metrics (for probabilities, mainly want calibration curve, logloss and brier score loss)

In [ ]:
train_xgb_evals.get_log_loss(), test_xgb_evals.get_log_loss()

In [ ]:
train_xgb_evals.get_brier_score_loss(), test_xgb_evals.get_brier_score_loss()

In [ ]:
train_xgb_evals.display_calibration_curve(nbins=20)
test_xgb_evals.display_calibration_curve(nbins=20)

In [ ]:
train_xgb_evals.get_auc_score(), test_xgb_evals.get_auc_score()

In [ ]:
train_xgb_evals.get_confusion_matrix(), test_xgb_evals.get_confusion_matrix()

In [ ]:
train_xgb_evals.display_confusion_matrix()
test_xgb_evals.display_confusion_matrix()

In [ ]:
train_xgb_evals.plot_roc_curve()
test_xgb_evals.plot_roc_curve()

In [ ]:
train_xgb_evals.plot_prauc_curve()
test_xgb_evals.plot_prauc_curve()

In [ ]:
train_xgb_evals.get_accuracy(), test_xgb_evals.get_accuracy()

In [ ]:
train_xgb_evals.get_recall(), test_xgb_evals.get_recall()

In [ ]:
train_xgb_evals.get_precision(), test_xgb_evals.get_precision()

In [ ]:
train_xgb_evals.get_f1_score(), test_xgb_evals.get_f1_score()